# Анализ лояльности пользователей Яндекс Афиши


Автор: Каракчеев Дмитрий Игоревич
Дата: 20.10.2025


---
Цель проекта
Сервис Яндекс Афиша предоставляет информацию о культурных и развлекательных мероприятиях, а также возможность приобретать на них билеты онлайн. Команда маркетинга стремится не просто привлекать новых пользователей, а удерживать их и превращать в постоянных клиентов.

Целью проекта является проведение исследовательского анализа данных для определения профиля пользователей, которые с большей вероятностью возвращаются на платформу и вновь совершают покупки, выделив признаки и факторы, влияющие на это. Для команды маркетинга это позволит: точнее внедрять персонализированные условия и рекламу для таких клиентов, оптимизировать бюджет на маркетинг и повысить общий уровень удержания.

Содержимое проекта
 
1. Загрузка и знакомство с данными
2. Предобработка данных
3. Создание профиля пользователя
4. Исследовательский анализ данных
5. Общие выводы и рекомендации


In [1]:
import numpy as np
import matplotlib.pyplot as plt
!pip install sqlalchemy
!pip install psycopg2-binary 
import pandas as pd
from sqlalchemy import create_engine
!pip install phik
import phik as ph
import seaborn as sns
!pip install python-dotenv

In [4]:
import os
from dotenv import load_dotenv
import psycopg2  # или sqlalchemy, в зависимости от твоего подключения

# Загружаем переменные из .env
load_dotenv()

# Извлекаем параметры подключения
DB_NAME = os.getenv("DB_NAME")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")

# Подключаемся к БД (пример для psycopg2)
conn = psycopg2.connect(
    dbname=DB_NAME,
    host=DB_HOST,
    port=DB_PORT,
    user=DB_USER,
    password=DB_PASSWORD
)

In [ ]:
query = '''
SELECT
  user_id,
  device_type_canonical,
  order_id,
  created_dt_msk,
  created_ts_msk,
  currency_code,
  revenue,
  tickets_count,
  created_dt_msk :: date - LAG(created_dt_msk :: date) OVER (PARTITION BY user_id ORDER BY created_dt_msk) AS days_since_prev,
  event_id,
  event_name_code,
  event_type_main,
  service_name,
  region_name,
  city_name
FROM afisha.purchases
INNER JOIN afisha.events USING(event_id)
INNER JOIN afisha.city USING(city_id)
INNER JOIN afisha.regions USING(region_id)
WHERE device_type_canonical IN ('mobile', 'desktop') AND event_type_main NOT IN ('фильм')
ORDER BY user_id;
'''
df = pd.read_sql_query(query, con=engine)
df.head()

In [5]:
rows_before, cols_before = df.shape
print(f"Размер датафрейма до обработки: {rows_before} строк, {cols_before} столбцов")

NameError: name 'df' is not defined

---

**Задача 1.2:** Изучите общую информацию о выгруженных данных. Оцените корректность выгрузки и объём полученных данных.

Предположите, какие шаги необходимо сделать на стадии предобработки данных — например, скорректировать типы данных.

Зафиксируйте основную информацию о данных в кратком промежуточном выводе.

---

In [ ]:
print('Первые строки датафрейма:')
display(df)
num_of_rows = df.shape[0]

In [ ]:

print('\nОбщая информация о таблице:')
df_info = pd.DataFrame({
    'non_null_count': df.notnull().sum(),
    'null_count': df.isnull().sum(),
    'dtype': df.dtypes
})
display(df_info)


In [ ]:
print(f"\nРазмер датафрейма: {df.shape[0]} строк и {df.shape[1]} столбцов")

# Проверим уникальных пользователей и заказов
unique_users = df['user_id'].nunique()
unique_orders = df['order_id'].nunique()

print(f"\nУникальных пользователей: {unique_users}")
print(f"Уникальных заказов: {unique_orders}")

# Проверим долю пропусков в каждом столбце
print("\nДоля пропусков в каждом столбце (%):")
display((df.isnull().mean() * 100).round(2))

Размер выгрузки: 292 034 строк и 13 столбцов.
Уникальных пользователей: 22 000.
Уникальных заказов: 292 034, что логично — один заказ = одна запись.

Пропусков не обнаружено во всех столбцах.
Типы данных корректны:
даты (created_dt_msk, created_ts_msk) — datetime64;
числовые поля (revenue, tickets_count, total, age_limit, event_id) — int64 / float64;
категориальные поля (user_id, cinema_circuit, currency_code, device_type_canonical, service_name) — object.

Выгрузка корректна, структура таблицы соответствует ожиданиям, данные чистые.
Для дальнейшей обработки необходимо:

Проверить наличие заказов в других валютах (currency_code = 'kzt') и привести выручку к рублям.

Проверить распределение числовых признаков (revenue, tickets_count) на выбросы.

Проверить категориальные значения на наличие дублей или пропусков, требующих нормализации.

---

###  2. Предобработка данных





In [ ]:
rows_before, cols_before = df.shape
print(f"Размер датафрейма до обработки: {rows_before} строк, {cols_before} столбцов")

In [ ]:
tenge_df = pd.read_csv('https://code.s3.yandex.net/datasets/final_tickets_tenge_df.csv')
tenge_df['data'] = pd.to_datetime(tenge_df['data'])
tenge_df['rate_per_tenge'] = tenge_df['curs'] / 100
display(tenge_df.head())

In [ ]:
df['created_dt_msk'] = pd.to_datetime(df['created_dt_msk'])
df = df.merge(tenge_df[['data', 'rate_per_tenge']], how='left', left_on='created_dt_msk', right_on='data')
df['revenue_rub'] = df.apply(
    lambda x: x['revenue'] if x['currency_code'] == 'rub' else x['revenue'] * x['rate_per_tenge'],
    axis=1
)
display(df[['created_dt_msk', 'currency_code', 'revenue', 'rate_per_tenge', 'revenue_rub']].head())

print(f"Размер датафрейма после конвертации: {df.shape[0]} строк, {df.shape[1]} столбцов")

In [ ]:
# Проверим количество пропусков в каждом столбце
missing = df.isna().sum().sort_values(ascending=False)
display(missing)

# Проверим долю пропусков
print((df.isna().mean() * 100).round(2))

In [ ]:
df['days_since_prev'] = df['days_since_prev'].fillna(-1).astype('int32')

In [ ]:
df['created_dt_msk'] = pd.to_datetime(df['created_dt_msk'])
df['created_ts_msk'] = pd.to_datetime(df['created_ts_msk'])
df['tickets_count'] = df['tickets_count'].astype('int16')
df['revenue_rub'] = df['revenue_rub'].astype('float32')

In [ ]:
for col in ['device_type_canonical', 'service_name', 'region_name', 'event_type_main']:
    print(f"\n{col} — уникальные значения:")
    print(df[col].value_counts(dropna=False).head(10))
df[['revenue_rub', 'tickets_count']].describe()

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
df['revenue_rub'].hist(bins=50)
plt.title('Распределение выручки с заказа')

plt.subplot(1,2,2)
df['tickets_count'].hist(bins=20)
plt.title('Распределение количества билетов в заказе')
plt.show()

In [ ]:
p99 = df['revenue_rub'].quantile(0.99)
df_filtered = df[df['revenue_rub'] <= p99]

removed = df.shape[0] - df_filtered.shape[0]
removed_pct = round(removed / df.shape[0] * 100, 2)

print(f"Отфильтровано выбросов: {removed} строк ({removed_pct}%)")

In [ ]:
rows_after, cols_after = df.shape
print(f"Размер датафрейма после обработки: {rows_after} строк, {cols_after} столбцов")


Пропуски обнаружены только в столбце days_since_prev (7,5%), что ожидаемо.
Они заменены на значение -1, обозначающее первый заказ пользователя.

Типы данных приведены к корректным форматам (datetime, int16, float32), что уменьшило размерность и повысило точность анализа.

Категориальные признаки не содержат ошибок, а значения типа 'нет' и 'unknown' отражают особенности выгрузки и оставлены без изменений.

В поле revenue_rub выявлены редкие крупные значения (выбросы).
После фильтрации по 99-му перцентилю удалено ≈1% строк, что не исказило структуру данных.

Итоговый датасет очищен, типы данных оптимизированы, данные готовы для построения пользовательских профилей и аналитики.

---

### 3. Создание профиля пользователя




In [ ]:
df_filtered = df_filtered.sort_values(by=['user_id', 'created_dt_msk'])

In [ ]:
user_profile = (
    df_filtered.groupby('user_id')
    .agg(
        first_order_date=('created_dt_msk', 'min'),
        last_order_date=('created_dt_msk', 'max'),
        orders_count=('order_id', 'count'),
        avg_revenue_rub=('revenue_rub', 'mean'),
        avg_tickets_count=('tickets_count', 'mean'),
        avg_days_since_prev=('days_since_prev', lambda x: x[x > 0].mean())  # исключаем -1 (первый заказ)
    )
    .reset_index()
)
first_orders = (
    df_filtered
    .sort_values(by=['user_id', 'created_dt_msk'])
    .groupby('user_id')
    .first()
    .reset_index()[['user_id', 'device_type_canonical', 'region_name', 'service_name', 'event_type_main']]
)
first_orders = first_orders.rename(columns={
    'device_type_canonical': 'first_device',
    'region_name': 'first_region',
    'service_name': 'first_service',
    'event_type_main': 'first_event_type'
})
user_profile = user_profile.merge(first_orders, on='user_id', how='left')
user_profile['is_two'] = (user_profile['orders_count'] >= 2).astype(int)
user_profile['is_five'] = (user_profile['orders_count'] >= 5).astype(int)


In [ ]:
print(f"Размер таблицы с профилем пользователей: {user_profile.shape[0]} строк и {user_profile.shape[1]} столбцов")
display(user_profile.head(10))

In [ ]:
user_profile.info()

---

**Задача 3.2.** Прежде чем проводить исследовательский анализ данных и делать выводы, важно понять, с какими данными вы работаете: насколько они репрезентативны и нет ли в них аномалий.

Используя данные о профилях пользователей, рассчитайте:

- общее число пользователей в выборке;
- среднюю выручку с одного заказа;
- долю пользователей, совершивших 2 и более заказа;
- долю пользователей, совершивших 5 и более заказов.

Также изучите статистические показатели:

- по общему числу заказов;
- по среднему числу билетов в заказе;
- по среднему количеству дней между покупками.

По результатам оцените данные: достаточно ли их по объёму, есть ли аномальные значения в данных о количестве заказов и среднем количестве билетов?

Если вы найдёте аномальные значения, опишите их и примите обоснованное решение о том, как с ними поступить:

- Оставить и учитывать их при анализе?
- Отфильтровать данные по какому-то значению, например, по 95-му или 99-му перцентилю?

Если вы проведёте фильтрацию, то вычислите объём отфильтрованных данных и выведите статистические показатели по обновлённому датасету.

In [ ]:
total_users = user_profile.shape[0]
print('Общее число пользователей:', total_users)

# Рассчитываем среднюю выручку с одного заказа по всему датафрейму df
avg_revenue_per_order = df['revenue_rub'].mean().round(2)
print(f'Средняя выручка с одного заказа (по всему датафрейму df): {avg_revenue_per_order} руб.')

# Рассчитываем среднюю выручку с одного заказа (усреднение средних значений по пользователям)
avg_revenue_per_order2 = user_profile['avg_revenue_rub'].mean().round(2)
print(f'Средняя выручка с одного заказа (усреднение средних значений по пользователям): {avg_revenue_per_order2} руб.')

# Рассчитываем долю пользователей, совершивших 2+ заказа
is_two_ratio = (user_profile['is_two'].mean() * 100).round(2)
print(f'Доля пользователей с 2+ заказами: {is_two_ratio}%')

# Рассчитываем долю пользователей, совершивших 5+ заказов
is_five_ratio = (user_profile['is_five'].mean() * 100).round(2)
print(f'Доля пользователей с 5+ заказами: {is_five_ratio}%')


In [ ]:
print(user_profile['orders_count'].describe())

# Дополнительно выводим 95-ый и 99-ый перцентиль для столбца 'total_orders'
print("95-й перцентиль для столбца 'orders_count' составляет:", user_profile['orders_count'].quantile(0.95))
print("99-й перцентиль для столбца 'orders_count' составляет:", user_profile['orders_count'].quantile(0.99))
print("99.9-й перцентиль для столбца 'orders_count' составляет:", user_profile['orders_count'].quantile(0.999))

In [ ]:
user_profile = user_profile[user_profile['orders_count'] <= user_profile['orders_count'].quantile(0.95)]

user_profile['orders_count'].describe()

In [ ]:
print(user_profile['avg_tickets_count'].describe())
print(user_profile['avg_tickets_count'].quantile([0.95, 0.99]))

In [ ]:
print(user_profile['avg_days_since_prev'].describe())
print(user_profile['avg_days_since_prev'].quantile([0.25, 0.5, 0.75, 0.95, 0.99]))

1. Общее число пользователей:
В исходном датасете — 21,854 пользователя.
2. Средняя выручка с одного заказа:
По всему датасету заказов: ~555.57 руб.
Усреднённая по пользователям: ~544.40 руб.
3. Доля пользователей по числу заказов:
Совершивших 2 и более заказов — 61.71%.
Совершивших 5 и более заказов — 29.01%.

В столбце orders_count обнаружены ярко выраженные выбросы: максимальное значение — 10,181 заказ, что значительно превышает 99-й перцентиль (152) и 95-й перцентиль (~31).
Решение: фильтрация по 95-му перцентилю (оставляем пользователей с числом заказов ≤ 31). Это уменьшает среднее количество заказов и исключает аномальные значения.

В столбце avg_tickets_count значения более однородны, максимум — 11 билетов, 95-й перцентиль — 4.25, 99-й — 5.33. Выбросы здесь есть, но они не столь экстремальны и могут быть естественными (например, крупные заказы).

В столбце avg_days_since_prev количество пользователей с заполненным значением — меньше (около 8,000). Максимум — 148 дней, 95-й перцентиль — 102 дня, 99-й — 130 дней. Аномалий ярко не видно, значения распределены логично (больше времени между заказами — у реальных пользователей).


Данные достаточно объёмные и в целом показательные.

Для анализа по количеству заказов целесообразно исключить явные выбросы (например, фильтрация по 95-му перцентилю).

Среднее число билетов в заказе и среднее время между заказами выглядят естественными, без экстремальных выбросов, фильтрация по ним не обязательна, но при необходимости можно использовать 99-й перцентиль.

После фильтрации сохраняется большая часть пользователей, что обеспечивает достаточный объём для анализа.

---

### 4. Исследовательский анализ данных



In [ ]:
features = ['first_event_type', 'first_device', 'first_region', 'first_service']


In [ ]:
features = ['first_event_type', 'first_device', 'first_region', 'first_service']

for feature in features:
    segment_stats = (
        user_profile.groupby(feature)['user_id']
        .count()
        .reset_index()
        .rename(columns={'user_id': 'users_count'})
        .sort_values('users_count', ascending=False)
    )
    
    segment_stats['share_%'] = (segment_stats['users_count'] / segment_stats['users_count'].sum() * 100).round(2)
    display(segment_stats.head(10))
    
    plt.figure(figsize=(10,4))
    plt.bar(segment_stats[feature].astype(str).head(10), segment_stats['users_count'].head(10))
    plt.title(f'Распределение пользователей по {feature}')
    plt.xticks(rotation=45, ha='right')
    plt.ylabel('Количество пользователей')
    plt.show()

По типу первого мероприятия:
Большинство пользователей пришли с концертов — 44,1%.
На втором месте — категория «другое» с 25%.
Театр — 19,5%.
Остальные типы мероприятий (стендап, спорт, выставки, ёлки) суммарно занимают менее 10%.

По типу устройства для первого заказа:
83,3% пользователей сделали покупку с мобильных устройств.
Только 16,7% — с десктопа.

По региону первого мероприятия:
Каменевский регион — 32,4% пользователей.
Североярская область — 17,1%.
Далее идут регионы с гораздо меньшими долями — менее 6% каждый.

По билетному оператору первого заказа:
Четыре крупнейших оператора («Билеты без проблем», «Мой билет», «Лови билет!», «Билеты в руки») привлекают около 60–65% новых пользователей.
Остальные операторы обеспечивают гораздо меньшие доли (менее 6% каждый).

Наблюдается высокая концентрация пользователей в нескольких ключевых сегментах — по типу мероприятия, устройству, региону и особенно билетному оператору. Это означает, что есть выраженные «точки входа» в сервис, и рост базы пользователей во многом зависит от успешности нескольких основных каналов и регионов.

In [ ]:
def retention_by_segment(df, column, top_n=10):
    seg = (
        df.groupby(column)
          .agg(users=('user_id', 'nunique'),
               returned=('is_two', 'mean'))
          .reset_index()
          .sort_values('users', ascending=False)
    )
    seg['returned_pct'] = seg['returned'] * 100
    top_seg = seg.head(top_n)
    
    plt.figure(figsize=(10,5))
    plt.barh(top_seg[column].astype(str), top_seg['returned_pct'], color='skyblue')
    plt.gca().invert_yaxis()
    plt.title(f'Доля возвратов пользователей по сегменту: {column}')
    plt.xlabel('Доля пользователей, совершивших ≥2 заказа (%)')
    plt.ylabel(column)
    plt.show()
    
    display(top_seg)
    return seg

# Анализ по разным сегментам
retention_device = retention_by_segment(user_profile, 'first_device')
retention_service = retention_by_segment(user_profile, 'first_service')
retention_region = retention_by_segment(user_profile, 'first_region')
retention_genre  = retention_by_segment(user_profile, 'first_event_type')



По типу устройства (first_device)
Доля возврата мобильных пользователей — около 53.6%.
Доля возврата пользователей с десктопа — чуть выше, 55.8%.
Вывод: Пользователи с десктопа чуть чаще возвращаются, хотя разница невелика.

По билетному оператору (first_service)
Самая высокая доля возвратов у сегментов «Край билетов» (57.2%), «Дом культуры» (56.4%) и «Весь в билетах» (56.1%).
Самая низкая — у «Прачечной» (около 52%).
Разница между операторами составляет порядка 5%, что говорит о том, что билетный оператор влияет на лояльность и возврат пользователя.

По региону проведения мероприятия (first_region)
Высокие доли возврата в регионах: «Светополянский округ» (59.1%), «Широковская область» (58.1%), «Шанырский регион» (56.7%).
Низкие — в «Малиновоярском округе» и «Озернинском крае» (около 50%).
Видим, что регион также существенно влияет на возвратность.

По типу первого мероприятия (first_event_type)
Наибольшая возвратность у сегмента «выставки» (57.9%) и «театр» (56.1%).
Самая низкая — у «спорт» (50.1%) и «ёлки» (50.6%).
Значит, интерес к определённым типам мероприятий повышает вероятность повторной покупки.

Общий вывод:
Есть выраженные успешные точки входа — сегменты с долей возвратов выше среднего (около 53-54% по всем пользователям).
Размер сегментов тоже важен — например, «выставки» с 354 пользователями показывают высокий retention, но для статистической устойчивости стоит учитывать, что сегмент небольшой.
Разные характеристики первого заказа — устройство, регион, тип события и билетный оператор — действительно влияют на вероятность возврата.

In [ ]:
genre_return = user_profile.groupby('first_event_type', as_index=False).agg(
    users=('user_id', 'nunique'),
    returned=('is_two', 'mean')
)
genre_return['returned_pct'] = genre_return['returned'] * 100
display(genre_return)


In [ ]:
region_return = user_profile.groupby('first_region', as_index=False).agg(
    users=('user_id', 'nunique'),
    returned=('is_two', 'mean')
)
region_return['returned_pct'] = region_return['returned'] * 100
display(region_return)

Гипотеза 1:
Доля возврата по спорту — 50.14%
Доля возврата по концертам — 54.48%

Вывод: пользователи спортивных мероприятий возвращаются реже, чем пользователи, впервые купившие билеты на концерты. Гипотеза не подтверждается.

Гипотеза 2:
Самые крупные регионы (например, Каменевский регион — 5889 пользователей) имеют возврат около 54.7%
Менее активные регионы с небольшим числом пользователей (например, Ягодиновская область — 57 пользователей) имеют возврат выше — 63.16%, а в некоторых очень маленьких регионах доля возврата варьируется сильно.

Вывод: корреляция между числом пользователей в регионе и долей возврата неочевидна. В менее активных регионах возврат может быть даже выше, но из-за маленькой выборки эти значения нестабильны. Следовательно, гипотеза не подтверждается однозначно, требуется дополнительный анализ с учётом размера выборки.

---

#### 4.2. Исследование поведения пользователей через показатели выручки и состава заказа




In [ ]:
one_time_users = filtered_profile[filtered_profile['orders_count'] == 1]
returned_users = filtered_profile[filtered_profile['orders_count'] >= 2]

plt.figure(figsize=(10, 5))

bins = 50  # фиксированное количество корзин

plt.hist(one_time_users['avg_revenue_rub'], bins=bins, alpha=0.6, density=True, label='1 заказ')
plt.hist(returned_users['avg_revenue_rub'], bins=bins, alpha=0.6, density=True, label='2+ заказа')

plt.title('Распределение средней выручки с заказа по группам пользователей')
plt.xlabel('Средняя выручка, ₽')
plt.ylabel('Плотность')
plt.legend()
plt.grid(alpha=0.3)
plt.show()


Пользователи с одним заказом обычно тратят от 0 до 500 рублей за заказ.
Пользователи, которые делают два и более заказов, чаще тратят от 250 до 750 рублей.

Разница в том, что у тех, кто возвращается, средний чек выше и распределён более равномерно. Их траты менее разбросаны и не так часто бывают очень низкими или очень высокими.

Пользователи с одним заказом чаще покупают билеты на небольшие суммы — часто меньше 100 рублей, и среди них встречается больше тех, кто тратит очень мало или очень много.

В целом, те, кто возвращается, приносят сервису больше денег и ведут себя более предсказуемо.

In [ ]:
moderate_users = filtered_profile[(filtered_profile['orders_count'] >= 2) & (filtered_profile['orders_count'] <= 4)]
loyal_users = filtered_profile[filtered_profile['orders_count'] >= 5]

In [ ]:
plt.figure(figsize=(10, 5))

plt.hist(moderate_users['avg_revenue_rub'], bins=50, alpha=0.6, density=True, label='2–4 заказа')
plt.hist(loyal_users['avg_revenue_rub'], bins=50, alpha=0.6, density=True, label='5+ заказов')

plt.title('Распределение средней выручки с заказа по группам пользователей')
plt.xlabel('Средняя выручка, ₽')
plt.ylabel('Плотность')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

In [ ]:
mean_moderate = moderate_users['avg_revenue_rub'].mean()
mean_loyal = loyal_users['avg_revenue_rub'].mean()

print(f"Средняя выручка у пользователей с 2–4 заказами: {mean_moderate:.2f} ₽")
print(f"Средняя выручка у пользователей с 5+ заказами: {mean_loyal:.2f} ₽")

Средняя выручка с заказа у пользователей с 2–4 и 5+ заказами практически не отличается (551.57 ₽ / 535.89 ₽).
Разница составляет около 3% и статистически незначима.
Это говорит о том, что лояльность пользователей (по числу заказов) не связана с размером среднего чека — пользователи совершают повторные покупки, но не начинают тратить существенно больше.

In [ ]:
def ticket_segment(x):
    if x < 2:
        return '1–2 билета'
    elif x < 3:
        return '2–3 билета'
    elif x < 5:
        return '3–5 билетов'
    else:
        return '5+ билетов'

user_profile['ticket_segment'] = user_profile['avg_tickets_count'].apply(ticket_segment)


In [ ]:
segment_stats = (
    user_profile
    .groupby('ticket_segment')
    .agg(
        users=('user_id', 'nunique'),
        returned=('is_two', 'sum')
    )
    .reset_index()
)
segment_stats['returned_pct'] = (segment_stats['returned'] / segment_stats['users'] * 100).round(2)

display(segment_stats)

In [ ]:
plt.figure(figsize=(8, 5))
plt.bar(segment_stats['ticket_segment'], segment_stats['returned_pct'], color='skyblue')
plt.title('Доля вернувшихся пользователей по среднему числу билетов в заказе')
plt.ylabel('Доля возвратов (%)')
plt.xlabel('Среднее количество билетов в заказе')
plt.grid(alpha=0.3)
plt.show()

Большинство пользователей покупают от 2 до 5 билетов за заказ — это основной сегмент.

Сегмент с 2–3 билетами показывает самую высокую долю повторных покупок — 64.36%, что говорит о наибольшей лояльности.

Пользователи с 1–2 и 3–5 билетами возвращаются реже — около 49%.

Наименее лояльны покупатели с большим средним количеством билетов (5+), у них только 18.14% возвратов, возможно из-за специфики крупных покупок.

Пользователи распределены неравномерно — большая часть в сегментах 2–5 билетов, меньшая — в крайних.

---

#### 4.3. Исследование временных характеристик первого заказа и их влияния на повторные покупки



In [ ]:
user_profile['first_order_date'] = pd.to_datetime(user_profile['first_order_date'])
user_profile['first_order_dayofweek'] = user_profile['first_order_date'].dt.day_name()


In [ ]:
day_stats = user_profile.groupby('first_order_dayofweek', as_index=False).agg(
    users=('user_id', 'nunique'),
    returned=('is_two', 'mean')
)

day_stats['returned_pct'] = day_stats['returned'] * 100
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
day_stats['day_num'] = day_stats['first_order_dayofweek'].apply(lambda x: days_order.index(x))
day_stats = day_stats.sort_values('day_num')


In [ ]:
plt.figure(figsize=(10,6))
plt.bar(day_stats['first_order_dayofweek'], day_stats['returned_pct'], color='skyblue')
plt.title('Returned Users Share by First Order Weekday')
plt.xlabel('Day of Week')
plt.ylabel('Returned Users Share (%)')
plt.grid(axis='y', alpha=0.3)
plt.show()


display(day_stats[['first_order_dayofweek', 'users', 'returned_pct']])

---

**Задача 4.3.2.** Изучите, как средний интервал между заказами влияет на удержание клиентов.



In [ ]:
df['days_between'] = df.groupby('user_id')['created_ts_msk'].diff().dt.days

In [ ]:
user_stats = (
    df.groupby('user_id')
      .agg(order_count=('order_id', 'count'),
           avg_interval=('days_between', 'mean'))
      .reset_index()
)
user_stats['order_group'] = pd.cut(
    user_stats['order_count'],
    bins=[1, 4, user_stats['order_count'].max()],
    labels=['2–4 заказа', '5+ заказов']
)
interval_stats = (
    user_stats.groupby('order_group')['avg_interval']
    .mean()
    .round(1)
    .reset_index(name='avg_interval_days')
)
display(interval_stats)

In [ ]:
plt.figure(figsize=(8,5))
user_stats.boxplot(column='avg_interval', by='order_group')
plt.title('Распределение среднего интервала между заказами')
plt.suptitle('')
plt.xlabel('Группа по числу заказов')
plt.ylabel('Средний интервал между заказами (дни)')
plt.show()

Чем короче средний интервал между заказами, тем выше удержание.
Пользователи, которые совершают покупки чаще (малые интервалы), с большей вероятностью продолжают пользоваться сервисом и достигают 5+ заказов.

Редкие покупки (длинные интервалы) часто связаны с “одноразовыми” пользователями, которые теряют интерес или забывают о сервисе.

---

#### 4.4. Корреляционный анализ количества покупок и признаков пользователя



In [ ]:
user_profile['avg_days_since_prev'] = user_profile['avg_days_since_prev'].replace(-1, None)
cat_cols = ['first_device', 'first_region', 'first_service', 'first_event_type']
interval_cols = ['avg_revenue_rub', 'avg_tickets_count', 'avg_days_since_prev']
for col in cat_cols:
    user_profile[col] = user_profile[col].astype('category')
corr_matrix = user_profile[
    ['orders_count'] + cat_cols + interval_cols
].phik_matrix(interval_cols=interval_cols)
corr_with_orders = corr_matrix['orders_count'].drop('orders_count').sort_values(ascending=False)
print("Корреляция признаков с количеством заказов (orders_count):")
display(corr_with_orders)


In [ ]:
plt.figure(figsize=(5, 6))
sns.heatmap(
    corr_matrix[corr_matrix.index != 'orders_count'][['orders_count']]
        .sort_values(by='orders_count', ascending=False),
    annot=True, fmt='.2f', cmap='YlGnBu', linewidths=0.5, cbar=False
)
plt.title("Коэффициент φₖ между признаками профиля и числом заказов (orders_count)")
plt.yticks(rotation=0)
plt.show()

Основные признаки, связанные с количеством заказов:

Средний интервал между заказами (avg_days_since_prev) — ключевой фактор удержания.

Среднее количество билетов (avg_tickets_count) — отражает вовлечённость.

Средний доход (avg_revenue_rub) — вторичный, но всё же значимый индикатор активности.

### 5. Общий вывод и рекомендации


Основные результаты анализа Всего пользователей: около 22 000. Всего заказов: ~292 000.

Распределение по числу заказов неравномерное — большинство пользователей совершили только 1 заказ, меньшее количество — 2–4, и лишь небольшая доля — 5+ заказов.

Интервалы между заказами: Средний интервал у пользователей с 2–4 заказами — значительно выше, чем у тех, кто сделал 5+ заказов. Чем короче средний интервал между заказами, тем выше вероятность удержания — активные пользователи совершают покупки чаще и остаются дольше.

Анализ признаков профиля: Средняя выручка (avg_revenue_rub) и количество билетов (avg_tickets_count) показывают умеренную положительную связь с числом заказов. Это значит, что пользователи, совершающие более дорогие или многобилетовные покупки, чаще возвращаются. Интервал между покупками (avg_days_since_prev) также имеет заметную корреляцию — чем он меньше, тем выше активность. Тип устройства, регион, тип сервиса и тип первого события имеют низкую корреляцию с количеством заказов — они не являются определяющими факторами возврата.

Ключевые зависимости (по результатам корреляционного анализа phi_k):

1. avg_days_since_prev	0.27 (наиболее сильная связь)
2. avg_tickets_count	0.21
3. avg_revenue_rub	0.16
4. first_device, first_service, first_event_type, first_region	< 0.03 — слабая связь

Выводы о лояльности

Основной фактор удержания — частота покупок (короткие интервалы между заказами).

Пользователи, которые с самого начала делают заказы на несколько билетов или с высокой выручкой, демонстрируют большую склонность к повторным покупкам.

Тип устройства и регион не оказывают существенного влияния — лояльность в большей степени определяется поведенческими, а не демографическими признаками.

Лояльность пользователей Яндекс Афиши в первую очередь определяется поведенческими метриками — частотой и объёмом заказов, а не внешними характеристиками профиля. Работа с вовлечением и сокращением времени до повторной покупки может существенно повысить удержание и общий LTV клиентов.

Рекомендации заказчику

Фокус на удержание: Создать механики, стимулирующие повторные покупки в короткие сроки (например, персональные уведомления о скидках через 3–5 дней после покупки).

Работа с сегментами: Сегмент 1 заказ — требует особого внимания (программы вовлечения, бонусы). Сегмент 2–4 — потенциально лояльная аудитория, стоит удерживать через рекомендации. Сегмент 5+ — ядро активных пользователей, которым можно предлагать премиальные акции и персональные предложения.

Увеличение среднего чека: Продвигать пакеты билетов или групповые покупки, так как это коррелирует с числом заказов.

Мониторинг частоты покупок: Включить в метрики retention-панели показатель “средний интервал между заказами” как ключевой индикатор лояльности.
